<a href="https://colab.research.google.com/github/tinetor/Covid/blob/master/fine_tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
raw_text = """
Q: What is Taylor Swift's full name?
A: Taylor Alison Swift.

Q: When was Taylor Swift born?
A: December 13, 1989.

Q: Where was Taylor Swift born?
A: West Reading, Pennsylvania.

Q: What was Taylor Swift's debut album called?
A: Taylor Swift.

Q: What year was her debut album released?
A: 2006.

Q: What was her first number-one hit on the Billboard Hot Country Songs chart?
A: Our Song.

Q: Which album features the hit song "Love Story"?
A: Fearless.

Q: What year did "Fearless" win the Grammy Award for Album of the Year?
A: 2010.

Q: How old was Taylor Swift when she won her first Album of the Year Grammy?
A: 20.

Q: What is the title of her 2010 album?
A: Speak Now.

Q: Which song from "Speak Now" is about a famous incident at an awards show?
A: Innocent.

Q: What is the name of her 2012 album?
A: Red.

Q: Which "Red" song is famously 10 minutes long in its re-recorded version?
A: All Too Well.

Q: What was Taylor Swift's first official pop album?
A: 1989.

Q: What year was "1989" released?
A: 2014.

Q: Which "1989" song includes the lyric "Are we out of the woods yet?"
A: Out of the Woods.

Q: What is the title of her 2017 album?
A: Reputation.

Q: Which song from "Reputation" begins with "My reputation's never been worse, so you must like me for me"?
A: Delicate.

Q: What is the name of her 2019 album?
A: Lover.

Q: Which "Lover" song is a collaboration with Brendon Urie?
A: ME!.

Q: What is the title of her first surprise album released in 2020?
A: Folklore.

Q: Which "Folklore" song tells the story from the perspective of a teenage girl named Betty?
A: Betty.

Q: What is the name of her second surprise album released in 2020?
A: Evermore.

Q: Which "Evermore" song is a collaboration with Bon Iver?
A: Evermore.

Q: What is the term for Taylor Swift's re-recordings of her earlier albums?
A: Taylor's Version.

Q: Which was the first album she re-recorded and released as "Taylor's Version"?
A: Fearless (Taylor's Version).

Q: What is the name of her 2022 album?
A: Midnights.

Q: Which "Midnights" track describes her as a "mastermind"?
A: Mastermind.

Q: What is the name of her ongoing concert tour?
A: The Eras Tour.

Q: Which city did The Eras Tour kick off in?
A: Glendale, Arizona (now known as Swift City for the duration of her concerts).

Q: What instrument is Taylor Swift most known for playing?
A: Guitar.

Q: What is the name of her cat that often appears in her social media?
A: Meredith Grey, Olivia Benson, and Benjamin Button.

Q: Which music video features many celebrity cameos and is a direct response to a feud?
A: Bad Blood.

Q: What is the name of her documentary released on Netflix in 2020?
A: Miss Americana.

Q: Which university awarded Taylor Swift an honorary doctorate?
A: New York University.

Q: What is the name of her fan base?
A: Swifties.

Q: What magazine named Taylor Swift their "Person of the Year" in 2023?
A: Time.

Q: What is the name of the short film she directed for "All Too Well (10 Minute Version)"?
A: All Too Well: The Short Film.

Q: Which artist did she co-write "This Is What You Came For" with?
A: Calvin Harris (under the pseudonym Nils Sjöberg).

Q: What is the title of her song about growing up and leaving home?
A: Fifteen.

Q: Which album features the song "You Belong With Me"?
A: Fearless.

Q: What is the name of the song she wrote about her mother's battle with cancer?
A: Soon You'll Get Better.

Q: Which of her albums features the song "Shake It Off"?
A: 1989.

Q: What is the title of her song that references a "picture to burn"?
A: Picture to Burn.

Q: Which artist did Taylor Swift bring on stage for a surprise duet during The Eras Tour for "Cardigan"?
A: Aaron Dessner.

Q: What is the title of her song about a magical first kiss?
A: Enchanted.

Q: Which "Red" song talks about a relationship that "got lost in translation"?
A: Holy Ground.

Q: What is the name of the song from "Lover" that discusses LGBTQ+ rights?
A: You Need To Calm Down.

Q: Which album features the song "Getaway Car"?
A: Reputation.

Q: What is the title of her song about feeling like an outcast in high school?
A: Teardrops on My Guitar.

Q: Which song from "Folklore" tells a love triangle from three different perspectives?
A: Cardigan, Betty, August.

Q: What is the name of the song she wrote with Ed Sheeran for "Red"?
A: Everything Has Changed.

Q: Which song features the lyric "long live the walls we crashed through"?
A: Long Live.

Q: What is the name of the song from "1989" that references "daylight"?
A: Daylight.

Q: Which album features the song "Dear John"?
A: Speak Now.

Q: What is the title of her song about a breakup during the holiday season?
A: Back to December.

Q: Which "Midnights" track is a collaboration with Lana Del Rey?
A: Snow On The Beach.

Q: What is the name of the song she wrote for the movie "The Hunger Games"?
A: Safe & Sound.

Q: Which album features the song "The Story of Us"?
A: Speak Now.

Q: What is the title of her song about a painful breakup that happened "all at once"?
A: The Moment I Knew.

Q: Which "Evermore" song is a collaboration with Haim?
A: No Body, No Crime.

Q: What is the name of the song from "Lover" that references "death by a thousand cuts"?
A: Death By A Thousand Cuts.

Q: Which album features the song "I Knew You Were Trouble."?
A: Red.

Q: What is the title of her song about a love interest who is "so gorgeous"?
A: Gorgeous.

Q: Which song from "Folklore" describes a summer romance that fades?
A: August.

Q: What is the name of her song about a painful goodbye set in a cafe?
A: Begin Again.

Q: Which album features the song "The Last Great American Dynasty"?
A: Folklore.

Q: What is the title of her song about a relationship where she feels like a "paper airplane"?
A: Out of the Woods.

Q: Which song from "Reputation" features the lyric "baby, let the games begin"?
A: ...Ready For It?.

Q: What is the name of the song about her ongoing feud with Kanye West?
A: Look What You Made Me Do.

Q: Which album features the song "The Archer"?
A: Lover.

Q: What is the title of her song about her own mortality and legacy?
A: The Best Day.

Q: Which "Midnights" track is about being a "vigilante shit"?
A: Vigilante Shit.

Q: What is the name of the song about a private jet and feeling like a "superstar"?
A: Invisible String.

Q: Which album features the song "Holy Ground"?
A: Red.

Q: What is the title of her song about wishing someone well after a breakup?
A: We Are Never Ever Getting Back Together.

Q: Which song from "Evermore" describes a murder mystery?
A: No Body, No Crime.

Q: What is the name of the song she wrote for her mother, Andrea Swift?
A: Soon You'll Get Better.

Q: Which album features the song "New Romantics"?
A: 1989 (Deluxe Edition).

Q: What is the title of her song about a powerful woman running a house?
A: The Last Great American Dynasty.

Q: Which song from "Lover" talks about wanting to marry someone?
A: Paper Rings.

Q: What is the name of her song about a first meeting with a significant other?
A: Mine.

Q: Which album features the song "Cold As You"?
A: Taylor Swift.

Q: What is the title of her song about moving on from a bad relationship?
A: Clean.

Q: Which "Midnights" track is about sleepless nights?
A: Anti-Hero.

Q: What is the name of the song that features the lyric "loving him was red"?
A: Red.

Q: Which album features the song "Haunted"?
A: Speak Now.

Q: What is the title of her song about a secret love affair?
A: Ivy.

Q: Which "Folklore" song is about a woman who poisons her husband?
A: Mad Woman.

Q: What is the name of the song about a fairytale romance?
A: Love Story.

Q: Which album features the song "Forever & Always"?
A: Fearless.

Q: What is the title of her song about a very public and scrutinized relationship?
A: The Man.

Q: Which "Reputation" track is about a renewed sense of confidence?
A: Look What You Made Me Do.

Q: What is the name of the song about reminiscing over a past love?
A: Come Back... Be Here.

Q: Which album features the song "Speak Now"?
A: Speak Now.

Q: What is the title of her song about forgiving someone who hurt her?
A: Innocent.

Q: Which "Midnights" track discusses a desire for stability?
A: Sweet Nothing.

Q: What is the name of the song that explores the darker side of fame?
A: tolerate it.

Q: Which album features the song "The Way I Loved You"?
A: Fearless.

Q: What is the title of her song about being heartbroken and alone?
A: Treacherous.

Q: Which "Evermore" song features the lyric "I come back stronger than a '90s trend"?
A: Willow.

Q: What is the name of her song about a childhood home?
A: Seven.

Q: Which album features the song "Begin Again"?
A: Red.

Q: What is the title of her song about an unrequited love?
A: You Belong With Me.
"""

In [ ]:
pip install  trl

In [ ]:
pip install bitsandbytes

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model_dir = 'Qwen/Qwen3-0.6B'
tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

model.config.use_cache = False
model.config.pretraining_tp = 1

In [ ]:
# prepare the model input
prompt = "When way Taylor Swift born?"
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
input_len = model_inputs["input_ids"].shape[1]  # acessa corretamente o comprimento dos ids de entrada
output_ids = generated_ids[0][input_len:].tolist()  # só a parte gerada
text = tokenizer.decode(output_ids, skip_special_tokens=True)  # decodifica só a geração

print(text)

<think>
Okay, the user is asking, "When way Taylor Swift born?" So, they want to know the date Taylor Swift was born. I need to check the current date to confirm. Let me recall, Taylor Swift was born on April 12, 2005. Wait, but I should make sure that's correct. Let me think... Yes, Taylor Swift was born on April 12, 2005. I should present that information clearly. Also, maybe mention her birthplace, which is in Canada. Oh, Canada, so the answer is April 12, 2005, Canada. I need to make sure the date is accurate. Double-checking, yes, that's correct. So the answer should be the date and the country.
</think>

Taylor Swift was born on **April 12, 2005**. She was born in Canada.


In [ ]:
from datasets import Dataset



# Quebra o texto em blocos de QA
pairs = raw_text.strip().split("\n\n")

# Converte para lista de dicionários com formato chat-style
data = []
for pair in pairs:
    lines = pair.strip().split("\n")
    if len(lines) >= 2:
        q = lines[0].replace("Q: ", "").strip()
        a = lines[1].replace("A: ", "").strip()
        data.append({
            "messages": [
                {"role": "user", "content": q},
                {"role": "assistant", "content": a}
            ]
        })

# Cria o Hugging Face Dataset
dataset = Dataset.from_list(data)
split = dataset.train_test_split(test_size=0.2)

train_subset = split["train"]
test_subset = split["test"]
# Visualiza o primeiro exemplo
print(dataset[0])


{'messages': [{'content': "What is Taylor Swift's full name?", 'role': 'user'}, {'content': 'Taylor Alison Swift.', 'role': 'assistant'}]}


In [ ]:
dataset

Dataset({
    features: ['messages'],
    num_rows: 104
})

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM , AutoTokenizer
from trl import SFTConfig, SFTTrainer,setup_chat_format
import torch
# Load dataset
from datasets import load_dataset, ReadInstruction
import os
from peft import LoraConfig
os.environ["WANDB_DISABLED"] = "true"

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"



# Carregar 1% do treino
##train_subset = load_dataset("HuggingFaceTB/smoltalk", "all", split=ReadInstruction("train", to=0.3, unit="%"))

# Carregar 1% do teste
##test_subset = load_dataset("HuggingFaceTB/smoltalk", "all", split=ReadInstruction("test", to=0.3, unit="%"))




# Setup chat template
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)



# TODO: Configure LoRA parameters
# r: rank dimension for LoRA update matrices (smaller = more compression)
rank_dimension = 6
# lora_alpha: scaling factor for LoRA layers (higher = stronger adaptation)
lora_alpha = 8
# lora_dropout: dropout probability for LoRA layers (helps prevent overfitting)
lora_dropout = 0.05

peft_config = LoraConfig(
    r=rank_dimension,  # Rank dimension - typically between 4-32
    lora_alpha=lora_alpha,  # LoRA scaling factor - typically 2x rank
    lora_dropout=lora_dropout,  # Dropout probability for LoRA layers
    bias="none",  # Bias type for LoRA. the corresponding biases will be updated during training.
    target_modules="all-linear",  # Which modules to apply LoRA to
    task_type="CAUSAL_LM",  # Task type for model architecture
)

# Configure trainer
training_args = SFTConfig(
    output_dir="./sft_output",
    max_steps=300,
    per_device_train_batch_size=4,
    learning_rate=5e-5,
    logging_steps=10,
    save_steps=100,
    eval_strategy="steps",
    eval_steps=50,
)

# Initialize trainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_subset,
    peft_config=peft_config,
    eval_dataset=test_subset,
    processing_class=tokenizer,
)

# Start training
trainer.train()

ValueError: Chat template is already added to the tokenizer. If you want to overwrite it, please set it to None

In [ ]:
messages = [{"role": "user", "content": "When was Taylor Swift born?"}]
input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)
outputs = model.generate(input_ids=input_ids, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
print(tokenizer.chat_template)  # se mostrar algo, já tem chat template aplicado


{%- if tools %}
    {{- '<|im_start|>system\n' }}
    {%- if messages[0].role == 'system' %}
        {{- messages[0].content + '\n\n' }}
    {%- endif %}
    {{- "# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>" }}
    {%- for tool in tools %}
        {{- "\n" }}
        {{- tool | tojson }}
    {%- endfor %}
    {{- "\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{\"name\": <function-name>, \"arguments\": <args-json-object>}\n</tool_call><|im_end|>\n" }}
{%- else %}
    {%- if messages[0].role == 'system' %}
        {{- '<|im_start|>system\n' + messages[0].content + '<|im_end|>\n' }}
    {%- endif %}
{%- endif %}
{%- set ns = namespace(multi_step_tool=true, last_query_index=messages|length - 1) %}
{%- for message in messages[::-1] %}
    {%- set index = (messages|length - 